In [113]:
from PIL import Image, ImageFont, ImageDraw, ImageFilter

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import random
import os
import csv
import json
import cv2
import numpy as np

In [114]:
## set default paths
font_path = "fonts"
base = "ita_passport_bg.png"
output_path = "fake_chars"
font_styles = os.listdir(font_path)
print(font_styles)
data_file = open("data/fake_chars.csv", "r", newline='')

['Hack-Italic.ttf', 'Hack-Regular.ttf', 'Hack-Bold.ttf', 'Hack-BoldItalic.ttf']


In [115]:
## preparing path
font_style = os.path.join(font_path, font_styles[2])

#loading font
font_size = 12
font = ImageFont.truetype(font_style, font_size)
char_width = 7

## color and coordinates definition
color = (0,0,0) #black

csv_reader = csv.reader(data_file)
# per studiare meglio il comportamento del codice, chiedo di effettuare le prossime operazioni solo sulla prima riga del file csv --> faccio un solo passaporto di prova
head_csv_reader = next(csv_reader)

pos_dict_list = []

In [116]:
passport_data = head_csv_reader
print("Passport Data: ", passport_data)

image = Image.open(base)


draw = ImageDraw.Draw(image)

color_facsimile = (255, 0, 0)
draw.text((10,0), "FACSIMILE    ", color_facsimile, font = font)

# forged_rnd mi definisce i tipi di errori da inserire nel passaporto che lo rendono falso
# random number: forged_rnd = 0 => elementi spuri nel passaporto ("passaporto sporco")
#                forged_rnd = 1 => lettere scritte una sopra l'altra
#                forged_rnd = 2 => caratteri in linee sfalsate   

forged_rnd = [random.randint(0,2),random.randint(0,2)]
print("Tipo di errori: ", forged_rnd)

# numero totale di caratteri di passport data
length_tot = 0
for i in range(len(passport_data)):
    length_tot += len(passport_data[i])
print("Numero di caratteri totali: ", length_tot)

if 2 in forged_rnd:
    # num_stagger_char numero di caratteri sfalsati da inserire nel testo --> max 32
    num_stagger_char = random.randint(1, 32)
    stagger_char_indexes = []
    for i in range(0, num_stagger_char):
        stagger_char_indexes.append(random.randint(0, length_tot - 1))
    print("Indici dei caratteri sfalsati: ", stagger_char_indexes)
        
fake_char = 0
current_char = 0
for i in range(len(passport_data)):
    for j in range(len(passport_data[i])):

            # 2 in forged_rnd: devo inserire caratteri sfalsati
            if 2 in forged_rnd and current_char in stagger_char_indexes:
                
                # FONT CHANGE
                c_fontstyle = font_style # it's here if we want it to dinamically change one day
                c_fontsize = font_size + random.randint(-2, 2)
                c_font = ImageFont.truetype(c_fontstyle, c_fontsize)

                # CALCOLO DELLE COORDINATE
                if i == 7: # la x per il settimo elemento è diversa (Luogo di nascita traslato di 87 pixels a destra)
                    coord_string = (190 + 87 + j * char_width + random.randint(-2,2), 100 + 31 * 4 + random.randint(-2,2))
                else:
                    coord_string = (190 + j * char_width + random.randint(-2,2), 100 + 31 * i + random.randint(-2,2))
            
                # SCRITTURA DEL TESTO
                if 1 in forged_rnd and passport_data[i][j]=='S' or passport_data[i][j]=='P' or passport_data[i][j]=='R':

                    draw.text(coord_string, passport_data[i][j], color, font = c_font)
                    draw.text(coord_string, 'B', color, font = c_font)

                    offset_x = random.randint(0,28)
                    offset_y = random.randint(0,char_width)
                    im_crop = image.crop((coord_string[0]-offset_x, coord_string[1]-offset_y, coord_string[0]-offset_x+32, coord_string[1]-offset_y+32))
                    file_name = "fake_char_"+str(fake_char)+".png"
                    im_crop.save(os.path.join(output_path, file_name), quality = 95)

                    fake_char += 1
                # solo 3 in forged_rnd 
                else:
                    draw.text(coord_string, passport_data[i][j], color, font = c_font)

                    offset_x = random.randint(0,28)
                    offset_y = random.randint(0,char_width)
                    im_crop = image.crop((coord_string[0]-offset_x, coord_string[1]-offset_y, coord_string[0]-offset_x+32, coord_string[1]-offset_y+32))
                    file_name = "fake_char_"+str(fake_char)+".png"
                    im_crop.save(os.path.join(output_path, file_name), quality = 95)

                    fake_char += 1
            else: # 3 not in forged_rnd
                
                # CALCOLO DELLE COORDINATE
                if i == 7:  # la x per il settimo elemento è diversa (Luogo di nascita traslato di 87 pixels a destra)
                    coord_string = (190 + 87 + j * char_width, 100 + 31 * 4)
                else:
                    coord_string = (190 + j * char_width, 100 + 31 * i)
                
                # 2 in forged_rnd --> sovrascrittura delle lettere
                if 1 in forged_rnd and  passport_data[i][j]=='S' or passport_data[i][j]=='P' or passport_data[i][j]=='R':
                    draw.text(coord_string, passport_data[i][j], color, font = font)
                    draw.text(coord_string, 'B', color, font = font)
                    
                    offset_x = random.randint(0,28)
                    offset_y = random.randint(0,char_width)
                    im_crop = image.crop((coord_string[0]-offset_x, coord_string[1]-offset_y, coord_string[0]-offset_x+32, coord_string[1]-offset_y+32))
                    file_name = "fake_char_"+str(fake_char)+".png"
                    im_crop.save(os.path.join(output_path, file_name), quality = 95)

                    fake_char += 1

                else:
                    draw.text(coord_string, passport_data[i][j], color, font = font)
            current_char += 1

# 0 in forged_rnd --> se devo inserire dei valori spuri, ne inserisco un numero randomico compreso tra 1 e 30
if 0 in forged_rnd:
    for i in range(random.randint(1,30)):
        # DEFINIZIONE COORDINATE
        coord_rnd_1 = (random.randint(0,600), random.randint(0,430))
        coord_rnd_2 = (coord_rnd_1[0]+1, coord_rnd_1[1])
        coord_rnd_3 = (coord_rnd_1[0]+1, coord_rnd_1[1]+1)
        coord_rnd_4 = (coord_rnd_1[0]+1, coord_rnd_1[1]+1)
        draw.polygon([coord_rnd_1, coord_rnd_2, coord_rnd_3, coord_rnd_4], fill='black', outline='black')

        offset = random.randint(0,28)
        im_crop = image.crop((coord_rnd_1[0]-offset, coord_rnd_1[1]-offset, coord_rnd_1[0]-offset+32, coord_rnd_1[1]-offset+32))
        file_name = "fake_char_"+str(fake_char)+".png"
        im_crop.save(os.path.join(output_path, file_name), quality = 95)

        fake_char += 1
print(fake_char)

Passport Data:  ['BARRACO', 'MARIO SALVATORE', 'ITALIANA', '8/11/1904', 'M', '6/13/2016', '6/13/2026', 'MILANO']
Tipo di errori:  [2, 0]
Numero di caratteri totali:  64
Indici dei caratteri sfalsati:  [41, 25, 60, 9, 43, 56, 61, 22]
38


In [117]:
# In alcune immagini andiamo ad inserire una minima rotazione, per evitare che gli unici passaporti leggibili siano quelli posizionati esattamente frontalmente alla telecamera (caso quasi impossibile...)
rand_img_to_rotate = random.randint(0, fake_char)
for i in range(rand_img_to_rotate):

    file_name = "fake_char_"+str(i)+".png"
    image = cv2.imread(os.path.join(output_path, file_name), cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    
    angle = random.randint(1,5)
    print("Angolo di rotazione: ", angle)

    # grab the dimensions of the image and then determine the centre
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = cv2.warpAffine(image, M, (nW, nH))
        
    cv2.imwrite(os.path.join(output_path, file_name), image)

Angolo di rotazione:  3
Angolo di rotazione:  4
Angolo di rotazione:  5
Angolo di rotazione:  4
Angolo di rotazione:  5
Angolo di rotazione:  5
Angolo di rotazione:  3
Angolo di rotazione:  4
Angolo di rotazione:  4
Angolo di rotazione:  1
Angolo di rotazione:  5
